In [1]:
# !python3 -m deeppavlov install squad_bert

### Imports and constants

In [1]:
import os
from torch import cuda, device

BASE_DIR = "./../"
DATA_DIR = os.path.join(BASE_DIR, "data/rucos")
SOURCES_DIR = os.path.join(BASE_DIR, "nti_ai_pipeline")
TRAIN_FILE_PATH = os.path.join(DATA_DIR, "rucos_train.jsonl")
VAL_FILE_PATH = os.path.join(DATA_DIR, "rucos_val.jsonl")
TEST_FILE_PATH = os.path.join(DATA_DIR, "rucos_test.jsonl")

BATCH_SIZE = 2
DEVICE = device("cuda" if cuda.is_available() else "cpu")
TRANSFORMER_NAME = "DeepPavlov/rubert-base-cased"

MOCK = True # turn off if run real pipeline, turn on if testing that things work

In [2]:
import sys
import importlib
import torch

sys.path.append(SOURCES_DIR)

import nti_rucos
import pipeline
from torch.utils import data as torch_data
from nti_rucos import modeling as rucos_modeling
from pipeline.saving import local_saver



my_modules = [nti_rucos, pipeline, rucos_modeling, local_saver]

for module in my_modules:
    importlib.reload(module)

### Loading data

In [3]:
container = nti_rucos.data.RucosContainer(TRAIN_FILE_PATH, VAL_FILE_PATH)

test_size = 0.001 if MOCK else 0.2
train_data, test_data = container.train_test_split(shuffle=True, test_size=test_size)

train_dataset = nti_rucos.data.RucosTrainDataset(train_data)
val_dataset = nti_rucos.data.RucosValDataset(test_data)

train_loader = torch_data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
val_loader = torch_data.DataLoader(val_dataset, batch_size=BATCH_SIZE)

### Pretraining model using MLM

In [ ]:
pretrainer = nti_rucos.pretraining.MLMBertPretrainer(checkpoints_dir="./pretrains")

In [ ]:
eval_every = 10 if MOCK else 20 * 1000
save_every = 10 if MOCK else 2 * 1000

path_to_best_pretrain_weights = pretrainer.pretrain(mname=TRANSFORMER_NAME, data_container=container,
                                                    batch_size=BATCH_SIZE, eval_every=eval_every, save_every=save_every)

### Defining model and trainer

In [4]:
proc = nti_rucos.modeling.RucosProcessor(TRANSFORMER_NAME, use_ner=False)
if MOCK:   
    model = rucos_modeling.mock_model.MockModel( # don't using SentPairBinaryClassifier because my laptop can't handle it with 12G of RAM
        use_ner=False
    )
else:
    model = rucos_modeling.SentPairBinaryClassifier(
        TRANSFORMER_NAME,
        droprate=0.2, 
        use_ner=False
    )
saver = pipeline.saving.local_saver.LocalSaver(save_dir='./saved_models')

manager = pipeline.modeling.ModelManager(model, proc, DEVICE, saver)

In [5]:
validator = nti_rucos.evaluating.RucosValidator()
weights_updater = nti_rucos.rucos_weights_updater.RucosWeightsUpdater(lr=7e-6, warmup=200, lr_end=1.7e-6, accum_iters=6, weight_decay=0.04, optimizer_class=torch.optim.AdamW) # NOTE: had set to 0 from 1e-2
trainer = pipeline.training.Trainer(validator, weights_updater)

### Running fit

In [6]:
# del model, manager
import gc, torch
gc.collect()
torch.cuda.empty_cache()

if MOCK:
    steps_betw_evals = 50
    max_step = 90
else:
    steps_betw_evals = 5000
    max_step = 50001
    
trainer.fit(train_loader, val_loader, manager, max_step=max_step, steps_betw_evals=steps_betw_evals, stop_patience=3)


_eval. Eval_value: 0.15068493783473969
Saved model. Eval value: 0.15068493783473969 Name: 1c246561-8637-4ec2-b263-e61f548a63ee


### Pseudo labeling

In [7]:
test_container = nti_rucos.data.RucosContainer(path=TEST_FILE_PATH, has_labels=False,
                                                  query_placeholder_union_mode="concatenate",
                                                  )
test_dataset = nti_rucos.data.RucosSubmDataset(test_container.get_data(), switch_texts=True)
if MOCK:
    test_dataset = torch_data.Subset(test_dataset, list(range(1000)))
    
test_loader = torch_data.DataLoader(test_dataset, batch_size=2)

In [8]:
predictor = nti_rucos.pseudo_labeling.ProbsDataPredictor()
pseudo_labeler = nti_rucos.pseudo_labeling.RegressionPseudoLabeler(predictor, chosen_proportion=0.15, 
                                                                            pos_to_neg_proportion=0.5)

In [9]:
pseudo_label_samples = pseudo_labeler.run(manager, test_loader)

In [10]:
pseudo_label_dataset = nti_rucos.data.RawSamplesDataset(pseudo_label_samples, switch_texts=True)
unioned_train_dataset = torch.utils.data.ConcatDataset([train_dataset, pseudo_label_dataset])

#### So this way we obtained pseudo labels by our model and can retrain it if needed using them

### Submitting

In [11]:
# best_manager = trainer.load_best_manager()
# best_manager = managing_model.ModelManager.load(saver, '9ab8822d-1d50-4d8d-904b-2e7a70dca78f')
best_manager = manager

submitter = nti_rucos.rucos_submitting.RucosSubmitter(subm_dir=os.path.join(BASE_DIR, 'submissions'))

# test loader is taken from Pseudo labeling step
submitter.create_submission(
    best_manager, test_loader, subm_file_name="subm_single_bert_50000_steps.jsonl",
    )


Some preds [SubmPred(text_id=tensor(0), probs=0.8425645365526541, start=tensor(54), end=tensor(62), placeholder='Приштины'), SubmPred(text_id=tensor(0), probs=0.32784725621175204, start=tensor(99), end=tensor(105), placeholder='Косово'), SubmPred(text_id=tensor(0), probs=14.337212342728614, start=tensor(332), end=tensor(353), placeholder='Югославской Федерации'), SubmPred(text_id=tensor(0), probs=-3.11626025698277, start=tensor(412), end=tensor(424), placeholder='Косовом поле'), SubmPred(text_id=tensor(0), probs=-0.2982896225016701, start=tensor(479), end=tensor(497), placeholder='Слободан Милошевич'), SubmPred(text_id=tensor(0), probs=1.3644815455689152, start=tensor(706), end=tensor(713), placeholder='Белград'), SubmPred(text_id=tensor(0), probs=0.48584214975597817, start=tensor(723), end=tensor(725), placeholder='ЕС'), SubmPred(text_id=tensor(0), probs=0.4643652464118247, start=tensor(816), end=tensor(822), placeholder='Сербии'), SubmPred(text_id=tensor(0), probs=1.3381478670657456